## Student Name: Venkateswarareddy Dundi  
## Student Email: Venkateswarareddy.Dundi-1@ou.edu

# Project 3: The Smart City Slicker

Imagine you are a stakeholder in a rising Smart City and want to know more about themes and concepts about existing smart cities. You also want to know where does your smart city place among others. In this project, you will perform 
exploratory data analysis, often shortened to EDA, to examine a data from the [2015 Smart City Challenge](https://www.transportation.gov/smartcity) to find facts about the data and communicating those facts through text analysis and visualizations.

In order to explore the data and visualize it, some modifications might need to be made to the data along the way. This is often referred to as data preprocessing or cleaning.
Though data preprocessing is technically different from EDA, EDA often exposes problems with the data that need to be fixed in order to continue exploring.
Because of this tight coupling, you have to clean the data as necessary to help understand the data.

In this project, you will apply your knowledge about data cleaning, machine learning, visualizations, and databases to explore smart city applications.

**Part 1** of the notebook will explore and clean the data. \
**Part 2** will take the results of the preprocessed data to create models and visualizations.

Empty cells are code cells. 
Cells denoted with [Your Answer Here] are markdown cells.
Edit and add as many cells as needed.

Output file for this notebook is shown as a table for display purposes. Note: The city name can be Norman, OK or OK Norman.

| city | raw text | clean text | clusterid | topicids | summary | keywords|
| -- | -- | -- | -- | -- | -- | -- |
|Norman, OK | Test, test , and testing. | test test test | 0 | T1, T2| test | test |

## Introduction
The Dataset: 2015 Smart City Challenge Applicants (non-finalist).
In this project you will use the applicant's PDFs as a dataset.
The dataset is from the U.S Department of Transportation Smart City Challenge.

On the website page for the data, you can find some basic information about the challenge. This is an interesting dataset. Think of the questions that you might be able to answer! A few could be:

1. Can I identify frequently occurring words that could be removed during data preprocessing?
2. Where are the applicants from?
3. Are there multiple entries for the same city in different applicantions?
4. What are the major themes and concepts from the smart city applicants?

Let's load the data!

## Loading and Handling files

Load data from `smartcity/`. 

To extract the data from the pdf files, use the [pypdf.pdf.PdfFileReader](https://pypdf.readthedocs.io/en/stable/index.html) class.
It will allow you to extract pages and pdf files and add them to a data structure (dataframe, list, dictionary, etc).
To install the module, use the command `pipenv install pypdf`.
You only need to handle PDF files, handling docx is not necessary.

In [6]:
import os
from pypdf import PdfReader
import pandas as pd
from collections import OrderedDict

#inp_path = './smartcity/'

inp_path = '{}/docs/smartcity/'.format(os.path.abspath(os.getcwd()))

files = os.listdir(inp_path)
files_content = OrderedDict()

#print(files)

Descriptions = []

for i in files:
    if i[-3:] == 'pdf':
        name = i.split('.')[0]
        pdf_reader = PdfReader(open(inp_path+i, 'rb'))
        s = ''
        for i in pdf_reader.pages:
            ##files_content.append(i.extract_text())
            s+=i.extract_text()

        #Cities.append(i[:-4])
        #Page_numbers.append(pdf_reader.pages)
        #Descriptions.append(s)
        files_content[name] = s
        

#print(files_content['GA Brookhaven'])

Create a data structure to add the city name and raw text. You can choose to split the city name from the file.

In [4]:
# This has been completed in the above cell. files_content is the dictionary containing both City Name and Raw text.

## Cleaning Up PDFs

One of the more frustrating aspects of PDF is loading the data into a readable format. The first order of business will be to preprocess the data. To start, you can use code provided by Text Analytics with Python, [Chapter 3](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch03%20-%20Processing%20and%20Understanding%20Text/Ch03a%20-%20Text%20Wrangling.ipynb): [contractions.py](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch05%20-%20Text%20Classification/contractions.py) (Pages 136-137), and [text_normalizer.py](https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch05%20-%20Text%20Classification/text_normalizer.py) (Pages 155-156). Feel free to download the scripts or add the code directly to the notebook (please note this code is performed on dataframes).

In addition to the data cleaning provided by the textbook, you will need to:
1. Consider removing terms that may effect clustering and topic modeling. Words to consider are cities, states, common words (smart, city, page, etc.). Keep in mind n-gram combinations are important; this can also be revisited later depending on your model's performance.
2. Check the data to remove applicants that text was not processed correctly. Do not remove more than 15 cities from the data.


In [7]:

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


import nltk
nltk.download('stopwords')
import spacy
import unicodedata
#from contractions import CONTRACTION_MAP
import re
from nltk.corpus import wordnet
import collections
#from textblob import Word
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
!pipenv run python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
# nlp_vec = spacy.load('en_vectors_web_lg', parse=True, tag=True, entity=True)



def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    if bool(soup.find()):
        [s.extract() for s in soup(['iframe', 'script'])]
        stripped_text = soup.get_text()
        stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    else:
        stripped_text = text
    return stripped_text


#def correct_spellings_textblob(tokens):
#	return [Word(token).correct() for token in tokens]  


def simple_porter_stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text


def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens


def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]|\[|\]' if not remove_digits else r'[^a-zA-Z\s]|\[|\]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=stopword_list):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_stemming=False, text_lemmatization=True, 
                     special_char_removal=True, remove_digits=True,
                     stopword_removal=True, stopwords=stopword_list):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)

        # remove extra newlines
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))

        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)

        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)

        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)

        # stem text
        if text_stemming and not text_lemmatization:
        	doc = simple_porter_stemming(doc)

        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove single characters which doesn't have any context
        doc = re.sub(r'\b\w\b', '', doc) 
        # remove 2 character-words which doesn't have any context
        doc = re.sub(r'\b\w{2}\b', '', doc)

         # lowercase the text    
        if text_lower_case:
            doc = doc.lower()

        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case, stopwords=stopwords)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

content_list = []
for k,v in files_content.items():
    content_list.append(v)

processed_list = normalize_corpus(content_list)


#print(processed_list)

[nltk_data] Downloading package stopwords to /home/gnani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### Add the cleaned text to the structure you created.


In [8]:
ordered_files_content = OrderedDict()

names = []

for i in files_content:
    names.append(i)


for i,j in zip(names, processed_list):
    ordered_files_content[i] = j

frames = OrderedDict()
frames['Cities'] = names
frames['Content'] = processed_list

df = pd.DataFrame(frames)

print(df)

               Cities                                            Content
0       GA Brookhaven  buford highway dekalb county dangerous stretch...
1      FL Tallahassee                                                   
2          NY Buffalo  department transportation notice funding oppor...
3        CA Riverside  city riverside california application departme...
4    AZ Scottsdale AZ  federal agency name department transportation ...
..                ...                                                ...
64         VA Norfolk  city norfolk response proposal usdot beyond tr...
65               DC_0  smart make district smart city submit response...
66         FL Orlando  beyond traffic smart city challenge vision nar...
67  VA Virginia Beach  project vision project demographics project al...
68           OK Tulsa  executive summary create vibrant city leverage...

[69 rows x 2 columns]


### Clean Up: Discussion
Answer the questions below.

#### Which Smart City applicants did you remove? What issues did you see with the documents?

I din't remove any city applicants. But there are lot of issues with the structures of the documents. The text around the images is being extracted at different parts of the document which doesn't maintain the consistency in the document.

#### Explain what additional text processing methods you used and why.

I used 2 additional preprocessing methods for cleaning the text. Upon using the provided scripts to process the text, there has been lot of words and single individual characters which doesn't have content. I believed that they doesn't add any value as efficient input to the model. So, I replaced the occurences of individual chacters and 2 letter characters with empty strings.

#### Did you identify any potientally problematic words?

There exists some words which contain special characters which splits the text into various parts upon cleaning. Some of those words didn;t have any proper context.

## Experimenting with Clustering Models

Now, you'll start to explore models to find the optimal clustering model. In this section, you'll explore [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), [Hierarchical](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html), and [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) clustering algorithms.
Create these algorithms with k_clusters for K-means and Hierarchical.
For each cell in the table provide the [Silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score), [Calinski and Harabasz score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score), and [Davies-Bouldin score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_score).

In each cell, create an array to store the values.
For example, 

|Algorithm| k = 9 | k = 18| k = 36 | Optimal k| 
|--|--|--|--|--|
|K-means| [S,CH,DB]| [S,CH,DB] | [S,CH,DB] | [S,CH,DB] |
|Hierarchical |[S,CH,DB]| [S,CH,DB]| [S,CH,DB] | [S,CH,DB]|
|DBSCAN | X | X | X | [S,CH,DB] |



### Optimality 
You will need to find the optimal k for K-means and Hierarchical algorithms.
Find the optimality for k in the range 2 to 50.
Provide the code used to generate the optimal k and provide justification for your approach.


|Algorithm| k = 9 | k = 18| k = 36 | Optimal k| 
|--|--|--|--|--|
|K-means|--|--|--|--|
|Hierarchical |--|--|--|--|
|DBSCAN | X | X | X | -- |



In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import pandas as pd


vectorizer = TfidfVectorizer()

# Fit and transform the text content to obtain the feature matrix
X = vectorizer.fit_transform(df['Content'])


# Perform k-means clustering with 9 clusters
kmeans = KMeans(n_clusters=35, random_state=42)          # Clusters = 9, 18, 36

kmeans.fit(X) 

# Calculate the evaluation metrics
silhouette = silhouette_score(X, kmeans.labels_)
calinski_harabasz = calinski_harabasz_score(X.toarray(), kmeans.labels_)
davies_bouldin = davies_bouldin_score(X.toarray(), kmeans.labels_)

print([round(silhouette, 6), round(calinski_harabasz, 6), round(davies_bouldin, 6)])

# Please find Below specified Hierarchial - Agglomerative clustering model


agg = AgglomerativeClustering(n_clusters=35)
agg.fit(X.toarray())

# Compute silhouette score, Calinski-Harabasz score, and Davies-Bouldin score
silhouette = silhouette_score(X.toarray(), agg.labels_)
calinski_harabasz = calinski_harabasz_score(X.toarray(), agg.labels_)
davies_bouldin = davies_bouldin_score(X.toarray(), agg.labels_)

print([round(silhouette, 6), round(calinski_harabasz, 6), round(davies_bouldin, 6)])

# Below specified the DB Scan model


"""  # Define the DBSCAN model
dbscan = DBSCAN(eps=15, min_samples=35)

# Fit the model to the data
dbscan.fit(X)

silhouette = silhouette_score(X, dbscan.labels_)
calinski_harabasz = calinski_harabasz_score(X.toarray(), dbscan.labels_)
davies_bouldin = davies_bouldin_score(X.toarray(), dbscan.labels_)

print([round(silhouette, 6), round(calinski_harabasz, 6), round(davies_bouldin, 6)]) """

/home/gnani/.local/share/virtualenvs/Project_3-p6KFvZ14/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[0.000658, 1.240011, 0.991234]
[-0.008831, 1.466335, 1.313887]


'  # Define the DBSCAN model\ndbscan = DBSCAN(eps=15, min_samples=35)\n\n# Fit the model to the data\ndbscan.fit(X)\n\nsilhouette = silhouette_score(X, dbscan.labels_)\ncalinski_harabasz = calinski_harabasz_score(X.toarray(), dbscan.labels_)\ndavies_bouldin = davies_bouldin_score(X.toarray(), dbscan.labels_)\n\nprint([round(silhouette, 6), round(calinski_harabasz, 6), round(davies_bouldin, 6)]) '

|Algorithm| k = 9 | k = 18| k = 36 | Optimal k = 35|
|--|--|--|--|--|
|K-means|[-0.07346, 1.29692, 1.52535]|[-0.01219, 1.179899, 1.16019]|[0.001504, 1.201422, 0.97888]|[0.000658, 1.240011, 0.991234]]|
|Hierarchical |[0.017623, 1.52721, 2.61455]|[-0.02055, 1.42741, 1.93749]|[-0.00486, 1.47445, 1.29274]|[-0.008831, 1.466335, 1.31388]|
|DBSCAN | X | X | X | X |



#### How did you approach finding the optimal k?

In General, To find the optimal K value there are 2 different strategies. They are : 1) Elbow method which used mean squared distance to find the K value. 2) Silhoutte Score analysis. I considered the silhouette score analysis to find the optimal K value. The value of the silhouette coefﬁcient is between [-1, 1] where a score of 1 denotes the best, meaning that the data points are very compact within the cluster to which it belongs and far away from the other clusters. So, I validated the K value for which we get maximum Silhouette value along with comparing the Other metrics score and considered it as optimal value.



    for i in range(1,51):

        kmeans = KMeans(n_clusters=i, random_state=42)          where Clusters = 1,2,3,4,5...,50

        kmeans.fit(X) 

        # Calculate the evaluation metrics
        silhouette = silhouette_score(X, kmeans.labels_)
        calinski_harabasz = calinski_harabasz_score(X.toarray(), kmeans.labels_)
        davies_bouldin = davies_bouldin_score(X.toarray(), kmeans.labels_)

        print(silhouette, calinski_harabasz, davies_bouldin)

#### What algorithm do you believe is the best? Why?

K-means performs better than the Hierarchial clustering due to its ability to cluster the objects of different shapes and sizes and better score.

### Add Cluster ID to output file
In your data structure, add the cluster id for each smart city respectively. Show the to append the clusterid code below.

In [10]:
df['Cluster Id'] = kmeans.labels_

print(df)

               Cities                                            Content   
0       GA Brookhaven  buford highway dekalb county dangerous stretch...  \
1      FL Tallahassee                                                      
2          NY Buffalo  department transportation notice funding oppor...   
3        CA Riverside  city riverside california application departme...   
4    AZ Scottsdale AZ  federal agency name department transportation ...   
..                ...                                                ...   
64         VA Norfolk  city norfolk response proposal usdot beyond tr...   
65               DC_0  smart make district smart city submit response...   
66         FL Orlando  beyond traffic smart city challenge vision nar...   
67  VA Virginia Beach  project vision project demographics project al...   
68           OK Tulsa  executive summary create vibrant city leverage...   

    Cluster Id  
0            9  
1           10  
2           34  
3           10  
4 

### Save Model

After finding the best model, it is desirable to have a way to persist the model for future use without having to retrain. Save the model using [model persistance](https://scikit-learn.org/stable/model_persistence.html). This model should be saved in the same directory as this notebook and should be loaded as the model for your `project3.py`.

Save the model as `model.pkl`. You do not have to use pickle, but be sure to save the persistance using one of the methods listed in the link.

In [11]:
X = vectorizer.fit_transform(df['Content'])


# Perform k-means clustering with 9 clusters
kmeans = KMeans(n_clusters=35, random_state=42)          # Clusters = 9, 18, 36

kmeans.fit(X)

from joblib import dump, load
dump(kmeans, 'model.pkl') 

/home/gnani/.local/share/virtualenvs/Project_3-p6KFvZ14/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['model.pkl']

In [15]:
X = df['Content']

from joblib import dump, load
dump(X, 'vector.pkl') 

['vector.pkl']

## Derving Themes and Concepts

Perform Topic Modeling on the cleaned data. Provide the top five words for `TOPIC_NUM = Best_k` as defined in the section above. Feel free to reference [Chapter 6](https://github.com/dipanjanS/text-analytics-with-python/tree/master/New-Second-Edition/Ch06%20-%20Text%20Summarization%20and%20Topic%20Models) for more information on Topic Modeling and Summarization.

### Extract themes
Write a theme for each topic (atleast a sentence each).

[Your Answer]

[Your Answer]

[Your Answer]

### Add Topid ID to output file
Add the top two topics for each smart city to the data structure.

## Gathering Applicant Summaries and Keywords

For each smart city applicant, gather a summary and keywords that are important to that document. You can use gensim to do this. Here are examples of functions that you could use.

```python

from gensim.summarization import summarize

def summary(text, ratio=0.2, word_count=250, split=False):
    return summarize(text, ratio= ratio, word_count=word_count, split=split)
    
from gensim.summarization import keywords

def keys(text, ratio=0.01):
    return keywords(text, ratio=ratio)
```

### Add Summaries and Keywords
Add summary and keywords to output file.

## Write output data

The output data should be written as a TSV file.
You can use `to_csv` method from Pandas for this if you are using a DataFrame.

`Syntax: df.to_csv('file.tsv', sep = '')` \
`df.to_csv('smartcity_eda.tsv', sep='\t')`

In [13]:
df.to_csv('smartcity_eda.tsv', sep='\t', index=False)

# This Displays the content of the dataframe in the form of (City Name, Content, Cluster ID)

In [14]:
print(df['Content'])

0     buford highway dekalb county dangerous stretch...
1                                                      
2     department transportation notice funding oppor...
3     city riverside california application departme...
4     federal agency name department transportation ...
                            ...                        
64    city norfolk response proposal usdot beyond tr...
65    smart make district smart city submit response...
66    beyond traffic smart city challenge vision nar...
67    project vision project demographics project al...
68    executive summary create vibrant city leverage...
Name: Content, Length: 69, dtype: object


# Moving Forward
Now that you have explored the dataset, take the important features and functions to create your `project3.py`.
Please refer to the project spec for more guidance.
